In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from autodm.llm import get_llm
from autodm.roll import Dice, DiceType
from autodm.character import Character
from autodm.locaitons import Location, LocationStore, setup_new_locations
from autodm.storyline import StoryLineWithStoryItems

from rich import print

/Users/michaelfrantz/miniconda3/envs/autodm/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from llama_index.core.program import LLMTextCompletionProgram
from llama_index.vector_stores.lancedb import LanceDBVectorStore
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from datetime import datetime
from llama_index.core import PromptTemplate, StorageContext, VectorStoreIndex, Document
from llama_index.core.node_parser import SimpleNodeParser
from pydantic import BaseModel, Field
from typing import Optional, List, Any, Union
from pathlib import Path
import torch

In [4]:
character = Character.generate(level=0)
print(character)

Character(
    chr_class='wizard',
    chr_race='human',
    name='Alicia',
    attributes=Attributes(strength=8, dexterity=13, constitution=13, intelligence=5, wisdom=6, charisma=11),
    level=0,
    max_hp=23,
    hp=23,
    spells=[],
    equipment=[]
)

In [5]:
locations = setup_new_locations()
print(locations)

{
    'The Eastern Frontier': Location(
        type='region',
        name='The Eastern Frontier',
        description='A vast and untamed region filled with diverse landscapes, from dense forests to rugged 
mountains. This area is home to various tribes and settlements that have built their lives around the natural 
resources provided by the land.',
        parent_name=None,
        parent_description=None
    ),
    'New Haven': Location(
        type='city',
        name='New Haven',
        description='A thriving settlement nestled in the heart of the Eastern Frontier, New Haven is known for its
bustling markets and welcoming atmosphere. The city has grown around a natural spring that provides fresh water to 
both humans and animals alike.',
        parent_name='The Eastern Frontier',
        parent_description='A vast and untamed region filled with diverse landscapes, from dense forests to rugged 
mountains. This area is home to various tribes and settlements that have built their lives around the natural 
resources provided by the land.'
    )
}

In [6]:
city, region = locations.current, locations.get(locations.current.parent_name)

In [7]:
story = StoryLineWithStoryItems.generate(character, city, region, locations)

In [8]:
VECTORSTORE_DIR = Path('~').expanduser() / '.autodm/vectorstore'
if not VECTORSTORE_DIR.exists():
    VECTORSTORE_DIR.mkdir(parents=True)

ADVENTURE_LOG_TABLE_NAME = 'adventure_log'

In [9]:
adventure_log_vectorstore = LanceDBVectorStore(uri=str(VECTORSTORE_DIR), table_name=str(ADVENTURE_LOG_TABLE_NAME))
storage_context = StorageContext.from_defaults(vector_store=adventure_log_vectorstore)


In [10]:
def get_device():
    if torch.backends.mps.is_available():
        return 'mps'
    elif torch.cuda.is_available():
        return 'cuda'
    else:
        return 'cpu'

In [30]:
class AdventureLog:
    def __init__(self, lance_uri=VECTORSTORE_DIR, lance_table=ADVENTURE_LOG_TABLE_NAME):
        self.lance_uri = lance_uri
        self.lance_table = lance_table
        self.storage_context = self._setup_storage_context()
        try:
            self.index = self._setup_index()
        except Exception as e:
            self.index = None
            print("Failed to set up index. Will try again on first log.", e)

    def _setup_storage_context(self):
        return StorageContext.from_defaults(
            vector_store=LanceDBVectorStore(
                uri=str(self.lance_uri), table_name=str(self.lance_table)
            )
        )

    def _setup_index(self, doc_or_docs: Union[Document, List[Document]] = None):
        embed_model = HuggingFaceEmbedding(
            "Alibaba-NLP/gte-base-en-v1.5", device=get_device(), trust_remote_code=True
        )
        if doc_or_docs is not None:
            if not isinstance(doc_or_docs, list):
                doc_or_docs = [doc_or_docs]
            return VectorStoreIndex.from_documents(
                doc_or_docs,
                embed_model=embed_model,
                # storage_context=self.storage_context,
            )
        else:
            return VectorStoreIndex(
                embed_model=embed_model, 
                # storage_context=self.storage_context
            )

    def add_entry(self, entry: str, user: str = "dm"):
        doc = Document(
            text=entry, extra_info={"user": user, "timestamp": str(datetime.now())}
        )
        if self.index is None:
            self.index = self._setup_index(doc)
        else:
            self.index.insert(doc)

    def search(self, query: str, top_k: int = 5):
        query_engine = self.index.as_query_engine(llm=get_llm(output_cls=str), similarity_top_k=top_k)
        return query_engine.query(query)

In [31]:
log = AdventureLog()

/Users/michaelfrantz/miniconda3/envs/autodm/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Failed to set up index. Will try again on first log. One of nodes, objects, or index_struct must be provided.

In [32]:
log.add_entry("The party enters the tavern.")

In [33]:
log.search("Where are we?")

Response(response='We are in a tavern.', source_nodes=[NodeWithScore(node=TextNode(id_='5ae36046-7ff8-4101-9518-2e629dd4196b', embedding=None, metadata={'user': 'dm', 'timestamp': '2024-06-01 18:28:03.559902'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='c9c46eba-6350-490d-81ed-c831e37f6c89', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'user': 'dm', 'timestamp': '2024-06-01 18:28:03.559902'}, hash='27871af6a7247ce2b75e65cb76928094ad0969b1258c4ebeec2de3547f411d7d')}, text='The party enters the tavern.', start_char_idx=0, end_char_idx=28, text_template='{metadata_str}\n\n{content}', metadata_template='{key}: {value}', metadata_seperator='\n'), score=0.28967875150610123)], metadata={'5ae36046-7ff8-4101-9518-2e629dd4196b': {'user': 'dm', 'timestamp': '2024-06-01 18:28:03.559902'}})